In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Practicum- ChauDao')

Mounted at /content/drive


In [ ]:
df_reg = pd.read_csv('./trained_models/Output/TO_CHAU/polyvore_vgg16.csv')
#top = pd.read_pickle(("./Polyvore/cat_v1/top_categories.pkl"))
#bottom = pd.read_pickle(("./Polyvore/cat_v1/bottom_categories.pkl"))
#df = pd.read_pickle("./Polyvore/cat_v1/df_full.pkl")

#top = pd.read_pickle(("./Polyvore/cat_v2/top_categories.pkl"))
#bottom = pd.read_pickle(("./Polyvore/cat_v2/bottom_categories.pkl"))
#df = pd.read_pickle("./Polyvore/cat_v2/df_full.pkl")

#for the first run, all these files will be created in the following steps

In [ ]:
top

## Prepare DFs 

###df_reg
mapping set_id with categories:
set_id, top (list), bottom (list)
set_id, top_id, bottom_id

#### top

In [ ]:
#cat_v1
#x = df_reg.loc[df_reg['cate1_name'] == 'upper-body', ('id','cate2_name','attr26_1_name','attr26_3_name','attr26_5_name','attr26_6_name')]
#dont use attribute columns that only have 1 value
#x['atts'] = x[['cate2_name','attr26_1_name','attr26_3_name','attr26_5_name','attr26_6_name']].values.tolist()

#cat_v2
x = df_reg.loc[df_reg['cate1_name'] == 'upper-body', ('id','cate2_name','texture_name','sleeve_part_name','shape_name','neckline_part_name','fabric_name','style_name')]
#dont use attribute columns that only have 1 value
x['atts'] = x[['cate2_name','texture_name','sleeve_part_name','shape_name','neckline_part_name','fabric_name','style_name']].values.tolist()

In [ ]:
data = []
x1 = pd.DataFrame(columns=['id', 'cat_name'])
for id in x.id.unique():
  attributes = x[(x.id == id)].atts.values[0]
  for i in range(0,len(attributes)):
    att = attributes[i]
    data.append({'id': id,'cat_name':att})
x1 = x1.append(data, ignore_index = True)

####bottom

In [ ]:
y = df_reg.loc[df_reg['cate1_name'] != 'upper-body', ('id','cate2_name','texture_name','sleeve_part_name','shape_name','neckline_part_name','fabric_name','style_name')]
#dont use attribute columns that only have 1 value
y['atts'] = y[['cate2_name','texture_name','sleeve_part_name','shape_name','neckline_part_name','fabric_name','style_name']].values.tolist()

# as there are more than 1 level of clothing item attributes, I decide to flatten the levels into 1 level only so that the model can be flexible with different ways of annotation.

In [ ]:
data = []
y1 = pd.DataFrame(columns=['id', 'cat_name'])
for id in y.id.unique():
  attributes = y[(y.id == id)].atts.values[0]
  for i in range(0,len(attributes)):
    att = attributes[i]
    data.append({'id': id,'cat_name':att})
y1 = y1.append(data, ignore_index = True)

In [ ]:
y1.head(1)
#in df  section, the column name 'cat_name' had been change to 'bottom'. Similar to x1.cat_name -> top

,id,cat_name
0,214181831/3.jpg,Shorts


In [ ]:
import numpy as np

In [ ]:
unique_top = x1.cat_name.unique()
top_ids = dict(zip(unique_top, np.arange(unique_top.shape[0], dtype=np.int32)))

unique_bottom = y1.cat_name.unique()
bottom_ids = dict(zip(unique_bottom, np.arange(unique_bottom.shape[0], dtype=np.int32)))

In [ ]:
bottom = pd.DataFrame(bottom_ids.items(), columns=['old', 'new'])
top = pd.DataFrame(top_ids.items(), columns=['old', 'new'])

In [ ]:
bottom

,old,new
0,Shorts,0
1,graphic,1
2,sleeveless,2
3,no_dress,3
4,no_neckline,4
5,cotton,5
6,conventional,6
7,solid,7
8,tight,8
9,denim,9


In [ ]:
top.to_pickle("./Polyvore/cat_v2/vgg16/top_categories.pkl")
bottom.to_pickle("./Polyvore/cat_v2/vgg16/bottom_categories.pkl")

####df

In [ ]:
x1['top_id'] = x1.cat_name.apply(lambda u: top_ids[u])
y1['bottom_id'] = y1.cat_name.apply(lambda m: bottom_ids[m])

In [ ]:
x1.head(5)

,id,cat_name,top_id
0,214181831/1.jpg,Shorts,0
1,214181831/1.jpg,solid,1
2,214181831/1.jpg,long_sleeve,2
3,214181831/1.jpg,no_dress,3
4,214181831/1.jpg,crew_neckline,4


In [ ]:
y1['set_id'] = y1['id'].str[:9]
x1['set_id'] = x1['id'].str[:9]

In [ ]:
x1.rename(columns = {'cat_name':'top'}, inplace = True)
y1.rename(columns = {'cat_name':'bottom'}, inplace = True)

In [ ]:
from tqdm import tqdm

In [ ]:
data = []
x2 = pd.DataFrame(columns = ['set_id','top'])
for set_id in tqdm(x1.set_id.unique()):
  top = x1[(x1.set_id == set_id)].top_id.unique()
  data.append({'set_id':set_id, 'top':top})

x2 = x2.append(data,ignore_index = True)

100%|██████████| 12433/12433 [01:58<00:00, 104.64it/s]


In [ ]:
x2

In [ ]:
data = []
y2 = pd.DataFrame(columns = ['set_id','bottom'])
for set_id in tqdm(y1.set_id.unique()):
  bottom = y1[(y1.set_id == set_id)].bottom_id.unique()
  data.append({'set_id':set_id, 'bottom':bottom})

y2 = y2.append(data,ignore_index = True)


100%|██████████| 10510/10510 [01:00<00:00, 172.84it/s]


In [ ]:
y2

,set_id,bottom
0,214181831,"[0, 1, 2, 3, 4, 5, 6]"
1,120161271,"[0, 7, 2, 3, 4, 5, 8]"
2,143656996,"[0, 7, 2, 3, 4, 5, 8]"
3,186627934,"[0, 7, 2, 3, 4, 5, 8]"
4,206969379,"[0, 1, 2, 3, 4, 5, 6]"
...,...,...
10505,100937795,"[0, 7, 2, 3, 4, 9, 6]"
10506,213533318,"[0, 7, 2, 3, 4, 5, 6]"
10507,216801059,"[0, 7, 2, 3, 4, 5, 6]"
10508,190488700,"[0, 7, 2, 3, 4, 5, 6]"


In [ ]:
#join 2 df
df = pd.merge(x2,y2,on='set_id')

In [ ]:
df

,set_id,top,bottom
0,214181831,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0, 1, 2, 3, 4, 5, 6]"
1,120161271,"[0, 7, 10, 3, 4, 5, 6]","[0, 7, 2, 3, 4, 5, 8]"
2,143656996,"[0, 1, 11, 3, 12, 5, 6, 2, 4]","[0, 7, 2, 3, 4, 5, 8]"
3,186627934,"[0, 1, 2, 3, 4, 5, 6, 7, 11]","[0, 7, 2, 3, 4, 5, 8]"
4,206969379,"[0, 1, 11, 3, 12, 5, 6]","[0, 1, 2, 3, 4, 5, 6]"
...,...,...,...
10203,100937795,"[0, 1, 2, 3, 4, 5, 6, 7, 11]","[0, 7, 2, 3, 4, 9, 6]"
10204,213533318,"[0, 14, 2, 3, 4, 5, 6, 1]","[0, 7, 2, 3, 4, 5, 6]"
10205,216801059,"[0, 1, 11, 3, 12, 5, 6]","[0, 7, 2, 3, 4, 5, 6]"
10206,190488700,"[0, 7, 2, 3, 4, 5, 6, 19]","[0, 7, 2, 3, 4, 5, 6]"


In [ ]:
#df.to_pickle("./Polyvore/cat_v1/df_full.pkl")
#df.to_pickle("./Polyvore/cat_v2/inceptionv3_v1/df_full.pkl")

df.to_pickle("./Polyvore/cat_v2/vgg16/df_full.pkl")

In [ ]:
#Notes: remember to check the number of set_id in df_full

### check number of set_id dont have top/ bottom

In [ ]:
#df=pd.read_pickle("./Polyvore/cat_v1/df_full.pkl")

In [ ]:
num_no_top = 0
num_no_bt = 0
for set_id in tqdm(df.set_id.unique()):
  t = df[(df.set_id == set_id)].top.values[0]
  len_top = len(t)
  b = df[(df.set_id == set_id)].bottom.values[0]
  len_bottom = len(b)
  if len_top == 0:
    num_no_top =num_no_top +1
  if len_bottom == 0:
    num_no_bt = num_no_bt+1

100%|██████████| 10208/10208 [00:27<00:00, 366.48it/s]


In [ ]:
num_no_top


0

In [ ]:
num_no_bt

0

##2. Edit df => structure: top, bottom in pair

### split into train + test set

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
df_train

,set_id,top,bottom
2280,177890155,"[10, 9, 2, 3, 4, 5, 6]","[0, 1, 2, 3, 9, 5, 6]"
7936,190378908,"[10, 9, 2, 3, 12, 5, 6, 7, 11, 4]","[0, 1, 2, 3, 9, 5, 6]"
5783,106444833,"[10, 7, 8, 3, 4, 5, 6, 9, 2]","[0, 1, 2, 3, 9, 5, 10]"
4515,210591329,"[10, 9, 2, 3, 4, 5, 6]","[0, 1, 2, 3, 9, 5, 6]"
4001,184429221,"[10, 9, 2, 3, 4, 5, 6, 7, 11]","[0, 1, 2, 3, 4, 5, 6]"
...,...,...,...
5226,215778055,"[10, 9, 8, 3, 4, 5, 6]","[0, 1, 2, 3, 9, 5, 6]"
5390,128047829,"[10, 1, 2, 3, 4, 5, 6, 8, 7]","[0, 1, 2, 3, 9, 5, 6]"
860,170348709,"[0, 7, 2, 3, 4, 5, 6]","[0, 12, 2, 3, 9, 5, 6]"
7603,206667483,"[10, 9, 8, 3, 12, 5, 6]","[0, 1, 2, 3, 9, 5, 6]"


In [ ]:
df_test

,set_id,top,bottom
5062,118070808,"[0, 14, 11, 16, 4, 5, 6]","[0, 7, 2, 3, 4, 9, 8]"
2405,209951913,"[0, 7, 2, 3, 4, 5, 6, 1, 13]","[0, 23, 2, 3, 4, 5, 6]"
9478,207942109,"[0, 9, 2, 3, 4, 5, 6, 1, 11, 12]","[0, 7, 2, 3, 4, 9, 6]"
8713,202127919,"[0, 1, 11, 3, 12, 5, 6, 7]","[0, 7, 2, 3, 4, 9, 6]"
9085,212038617,"[0, 14, 2, 3, 4, 5, 6]","[0, 13, 2, 3, 4, 9, 8]"
...,...,...,...
7464,206623682,"[0, 1, 2, 3, 4, 5, 6, 14, 13]","[0, 13, 2, 3, 4, 5, 6]"
4248,203955931,"[0, 1, 2, 3, 4, 5, 6]","[0, 7, 2, 3, 4, 5, 6]"
8920,216594724,"[0, 7, 11, 3, 12, 5, 6, 1, 2, 13]","[0, 7, 2, 3, 4, 9, 6]"
2388,130957723,"[0, 1, 11, 3, 12, 5, 6]","[0, 7, 11, 3, 12, 5, 6]"


### split each top to each bottom

In [ ]:
import itertools  

#train 
df_train_split = pd.DataFrame() 


for set_id in tqdm(df_train.set_id.unique()):
  #x = df[df['set_id'] == set_id].index
  A = df_train.loc[df_train['set_id'] == set_id, 'top'].values[0]
  B = df_train.loc[df_train['set_id'] == set_id, 'bottom'].values[0]
  c = list(itertools.product(A, B))
  temp = pd.DataFrame(c,columns =['top_id', 'bottom_id'])
  temp['set_id'] = set_id
  df_train_split = df_train_split.append(temp)

    

100%|██████████| 8166/8166 [01:06<00:00, 122.20it/s]


In [ ]:
df_train_split.head(5)

,top_id,bottom_id,set_id
0,10,0,177890155
1,10,1,177890155
2,10,2,177890155
3,10,3,177890155
4,10,9,177890155


In [ ]:
#df_train_split.rename(columns = {'top':'top_id'}, inplace = True)
#df_train_split.rename(columns = {'bottom':'bottom_id'}, inplace = True)

In [ ]:
#test 
df_test_split = pd.DataFrame() 


for set_id in tqdm(df_test.set_id.unique()):
  #x = df[df['set_id'] == set_id].index
  A = df_test.loc[df_test['set_id'] == set_id, 'top'].values[0]
  B = df_test.loc[df_test['set_id'] == set_id, 'bottom'].values[0]
  c = list(itertools.product(A, B))
  temp = pd.DataFrame(c,columns =['top_id', 'bottom_id'])
  temp['set_id'] = set_id
  df_test_split = df_test_split.append(temp)

100%|██████████| 2042/2042 [00:07<00:00, 266.72it/s]


In [ ]:
df_test_split.head(5)

,top_id,bottom_id,set_id
0,0,0,118070808
1,0,7,118070808
2,0,2,118070808
3,0,3,118070808
4,0,4,118070808


In [ ]:
#df_train_split.to_pickle("./Polyvore/cat_v1/train/df_train_split.pkl")
#df_test_split.to_pickle("./Polyvore/cat_v1/test/df_test_split.pkl")

df_train_split.to_pickle("./Polyvore/cat_v2/vgg16/train/df_train_split.pkl")
df_test_split.to_pickle("./Polyvore/cat_v2/vgg16/test/df_test_split.pkl")

## 3. Create file df_triplest

### df_triplest

In [ ]:
#top = pd.read_pickle(("./Polyvore/cat_v1/top_categories.pkl"))
#bottom = pd.read_pickle(("./Polyvore/cat_v1/bottom_categories.pkl"))

top = pd.read_pickle(("./Polyvore/cat_v2/vgg16/top_categories.pkl"))
bottom = pd.read_pickle(("./Polyvore/cat_v2/vgg16/bottom_categories.pkl"))

In [ ]:
#train
df_train_triplest = pd.DataFrame(columns=['set_id', 'positive_item_id', 'negative_item_id'])
# set_id = top_id; positive_item_id = positive_bottom_id, negative_item_id = negative_item_id

cat = list(bottom["new"]) #np.arange(0,14, 1)
data = []

for top_id in tqdm(df_train_split.top_id.unique()):
  positive_items = df_train_split[(df_train_split.top_id == top_id)].bottom_id.unique()

  pos = positive_items[:]
  negative_items = cat
  for i in range(len(pos)):
      element = pos[i]
      c = negative_items
      negative_items = [c[k] for k in range(len(c)) if c[k] != element ]

  data.append({'set_id': top_id, 'positive_item_id': positive_items[:], 'negative_item_id': negative_items})
df_train_triplest = df_train_triplest.append(data, ignore_index=True)



100%|██████████| 29/29 [00:00<00:00, 341.76it/s]


In [ ]:
df_train_triplest.shape

(29, 3)

In [ ]:
# number of set that dont have negative/ positive items
num_no_pos = 0
num_no_neg = 0
for set_id in tqdm(df_train_triplest.set_id.unique()):
  t = df_train_triplest[(df_train_triplest.set_id == set_id)].negative_item_id.values[0]
  len_neg = len(t)
  b = df_train_triplest[(df_train_triplest.set_id == set_id)].positive_item_id.values[0]
  len_pos = len(b)
  if len_neg == 0:
    num_no_neg =num_no_neg +1
  if len_pos == 0:
    num_no_pos = num_no_pos+1

100%|██████████| 29/29 [00:00<00:00, 822.60it/s]


In [ ]:
num_no_neg
#inceptionv3_v1: 15/40 tops can go with all bottom => trouble
#resnet50: 9/19 tops can go with all bottom => trouble
#vgg16: 7/29 tops can go with all bottom

7

In [ ]:
#test
df_test_triplest = pd.DataFrame(columns=['set_id', 'positive_item_id', 'negative_item_id'])
# set_id = top_id; positive_item_id = positive_bottom_id, negative_item_id = negative_item_id

cat = list(bottom["new"]) #np.arange(0,14, 1)
data = []

for top_id in tqdm(df_test_split.top_id.unique()):
  positive_items = df_test_split[(df_test_split.top_id == top_id)].bottom_id.unique()

  pos = positive_items[:]
  negative_items = cat
  for i in range(len(pos)):
      element = pos[i]
      c = negative_items
      negative_items = [c[k] for k in range(len(c)) if c[k] != element ]

  data.append({'set_id': top_id, 'positive_item_id': positive_items[:], 'negative_item_id': negative_items})
df_test_triplest = df_test_triplest.append(data, ignore_index=True)



100%|██████████| 30/30 [00:00<00:00, 704.86it/s]


In [ ]:
df_test_triplest

In [ ]:
# mark tops that dont have negative/ positive bottom items => should I remove these tops?
#train
num_no_pos = 0
num_no_neg = 0
for set_id in tqdm(df_train_triplest.set_id.unique()):
  t = df_train_triplest[(df_train_triplest.set_id == set_id)].negative_item_id.values[0]
  b = df_train_triplest[(df_train_triplest.set_id == set_id)].positive_item_id.values[0]
  index = df_train_triplest[(df_train_triplest.set_id == set_id)].index
  if len(t) == 0:
    num_no_neg =num_no_neg +1
    df_train_triplest.at[index,'no_neg_flag'] = 1 
  else:
    df_train_triplest.at[index,'no_neg_flag'] = 0 

  if len(b) == 0:
    num_no_pos = num_no_pos+1
    df_train_triplest.at[index,'no_pos_flag'] = 1 
  else:
    df_train_triplest.at[index,'no_pos_flag'] = 0

100%|██████████| 29/29 [00:00<00:00, 379.67it/s]


In [ ]:
df_train_triplest.head(5)

,set_id,positive_item_id,negative_item_id,no_neg_flag,no_pos_flag
0,10,"[0, 1, 2, 3, 9, 5, 6, 10, 4, 8, 11, 14, 12, 13...",[],1.0,0.0
1,9,"[0, 1, 2, 3, 9, 5, 6, 10, 4, 8, 11, 14, 12, 13...",[],1.0,0.0
2,2,"[0, 1, 2, 3, 9, 5, 6, 10, 4, 8, 11, 12, 13, 14...",[],1.0,0.0
3,3,"[0, 1, 2, 3, 9, 5, 6, 10, 4, 8, 11, 14, 12, 13...",[],1.0,0.0
4,4,"[0, 1, 2, 3, 9, 5, 6, 10, 4, 8, 11, 14, 12, 13...",[],1.0,0.0


In [ ]:
#test
num_no_pos = 0
num_no_neg = 0
for set_id in tqdm(df_test_triplest.set_id.unique()):
  t = df_test_triplest[(df_test_triplest.set_id == set_id)].negative_item_id.values[0]
  b = df_test_triplest[(df_test_triplest.set_id == set_id)].positive_item_id.values[0]
  index = df_test_triplest[(df_test_triplest.set_id == set_id)].index
  if len(t) == 0:
    num_no_neg =num_no_neg +1
    df_test_triplest.at[index,'no_neg_flag'] = 1 
  else:
    df_test_triplest.at[index,'no_neg_flag'] = 0 

  if len(b) == 0:
    num_no_pos = num_no_pos+1
    df_test_triplest.at[index,'no_pos_flag'] = 1 
  else:
    df_test_triplest.at[index,'no_pos_flag'] = 0

100%|██████████| 30/30 [00:00<00:00, 374.83it/s]


In [ ]:

#df_train_triplest.to_pickle("./Polyvore/cat_v1/train/df_train_triplest.pkl")
#df_test_triplest.to_pickle("./Polyvore/cat_v1/test/df_test_triplest.pkl")

df_train_triplest.to_pickle("./Polyvore/cat_v2/vgg16/train/df_train_triplest.pkl")
df_test_triplest.to_pickle("./Polyvore/cat_v2/vgg16/test/df_test_triplest.pkl")

In [ ]:
#all data, just for check code: df_train_triplest.to_pickle(("./Polyvore/cat_v1/df_triplest.pkl"))

### df_triplest fin

In [ ]:
import itertools  

df_train_triplest_fin = pd.DataFrame() 

for set_id in tqdm(df_train_triplest.set_id.unique()):
  #x = df[df['set_id'] == set_id].index
  A = df_train_triplest.loc[df_train_triplest['set_id'] == set_id, 'positive_item_id'].values[0]
  B = df_train_triplest.loc[df_train_triplest['set_id'] == set_id, 'negative_item_id'].values[0]
  c = list(itertools.product(A, B))
  temp = pd.DataFrame(c,columns =['positive_item_id', 'negative_item_id'])
  temp['set_id'] = set_id
  df_train_triplest_fin = df_train_triplest_fin.append(temp)

100%|██████████| 29/29 [00:00<00:00, 331.64it/s]


In [ ]:
df_train_triplest_fin#.head(5) 
#v1: 2324 rows
#inceptionv3_v1:  4038 rows
#resnet50: 645 rows
#vgg16: 3326 rows

,positive_item_id,negative_item_id,set_id
0,0,27,7
1,0,28,7
2,7,27,7
3,7,28,7
4,2,27,7
...,...,...,...
185,12,24,25
186,12,25,25
187,12,26,25
188,12,27,25


In [ ]:
df_train_triplest_fin.set_id.unique()

array([ 7, 11, 13, 10,  8, 12, 20, 14, 23, 18, 19, 15,  9, 16, 27, 21, 26,
       17, 24, 28, 22, 25])

In [ ]:
#df_train_triplest_fin.to_pickle(("./Polyvore/cat_v1/train/df_train_triplest_fin.pkl"))
#./Polyvore/cat_v1/train/df_train_triplest.pkl

df_train_triplest_fin.to_pickle(("./Polyvore/cat_v2/vgg16/train/df_train_triplest_fin.pkl"))

In [ ]:
import itertools  

df_test_triplest_fin = pd.DataFrame() 

for set_id in tqdm(df_test_triplest.set_id.unique()):
  #x = df[df['set_id'] == set_id].index
  A = df_test_triplest.loc[df_test_triplest['set_id'] == set_id, 'positive_item_id'].values[0]
  B = df_test_triplest.loc[df_test_triplest['set_id'] == set_id, 'negative_item_id'].values[0]
  c = list(itertools.product(A, B))
  temp = pd.DataFrame(c,columns =['positive_item_id', 'negative_item_id'])
  temp['set_id'] = set_id
  df_test_triplest_fin = df_test_triplest_fin.append(temp)


100%|██████████| 30/30 [00:00<00:00, 410.73it/s]


In [ ]:
df_test_triplest_fin 
#v1: 2504 rows
#inceptionv3_v1:  7976 rows
#resnet50: 916 rows
#vgg16: 4666 rows

,positive_item_id,negative_item_id,set_id
0,0,21,0
1,0,24,0
2,0,27,0
3,0,28,0
4,7,21,0
...,...,...,...
149,6,24,28
150,6,25,28
151,6,26,28
152,6,27,28


In [ ]:
#df_train_triplest_fin.to_pickle("./Polyvore/cat_v1/train/df_train_triplest_fin.pkl")
#df_test_triplest_fin.to_pickle("./Polyvore/cat_v1/test/df_test_triplest_fin.pkl")

df_train_triplest_fin.to_pickle("./Polyvore/cat_v2/vgg16/train/df_train_triplest_fin.pkl")
df_test_triplest_fin.to_pickle("./Polyvore/cat_v2/vgg16/test/df_test_triplest_fin.pkl")

## 4. Build model

In [ ]:
#top = pd.read_pickle(("./Polyvore/cat_v1/top_categories.pkl"))
#bottom = pd.read_pickle(("./Polyvore/cat_v1/bottom_categories.pkl"))

top = pd.read_pickle(("./Polyvore/cat_v2/vgg16/top_categories.pkl"))
bottom = pd.read_pickle(("./Polyvore/cat_v2/vgg16/bottom_categories.pkl"))

In [ ]:
#df_train_triplest_fin = pd.read_pickle("./Polyvore/cat_v1/train/df_train_triplest_fin.pkl")
#df_test_triplest_fin = pd.read_pickle("./Polyvore/cat_v1/test/df_test_triplest_fin.pkl")

df_train_triplest_fin = pd.read_pickle("./Polyvore/cat_v2/vgg16/train/df_train_triplest_fin.pkl")

In [ ]:
#frames = [df_train_triplest_fin, df_test_triplest_fin]
  
#df_triplest = pd.concat(frames)
df_triplest = df_train_triplest_fin

In [ ]:
df_triplest.drop_duplicates()
df_triplest

,positive_item_id,negative_item_id,set_id
0,0,27,7
1,0,28,7
2,7,27,7
3,7,28,7
4,2,27,7
...,...,...,...
185,12,24,25
186,12,25,25
187,12,26,25
188,12,27,25


In [ ]:
unique_users = top.new.unique()
unique_movies = bottom.new.unique() #np.arange(0,34, 1) 33 category

num_users = unique_users.shape[0]
num_items = unique_movies.shape[0]

num_users, num_items

(30, 29)

In [ ]:
unique_movie_ids = list(bottom.new.unique())

In [ ]:
import numpy as np
import pandas as pd
from os import path
from collections import OrderedDict
from tqdm import tqdm
from typing import Dict

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Embedding, Flatten, Input, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def bpr_predict(model: Model, user_id: int, item_ids: list, user_layer='user_embedding', item_layer='item_embedding'):
    """
    Predict by multiplication user vector by item matrix
    
    :return: list of the scores
    """
    user_vector = model.get_layer(user_layer).get_weights()[0][user_id]
    item_matrix = model.get_layer(item_layer).get_weights()[0][item_ids]

    scores = (np.dot(user_vector, item_matrix.T))

    return scores

In [ ]:
@tf.function
def identity_loss(_, y_pred):
    return tf.math.reduce_mean(y_pred)

In [ ]:
@tf.function
def bpr_triplet_loss(X: dict):
    """
    Calculate triplet loss - as higher the difference between positive interactions
    and negative interactions as better

    :param X: X contains the user input, positive item input, negative item input
    :return:
    """
    positive_item_latent, negative_item_latent, user_latent = X

    positive_interactions = tf.math.reduce_sum(tf.math.multiply(user_latent, positive_item_latent), axis=-1, keepdims=True)
    negative_interactions = tf.math.reduce_sum(tf.math.multiply(user_latent, negative_item_latent), axis=-1, keepdims=True)

    return tf.math.subtract(tf.constant(1.0), tf.sigmoid(tf.math.subtract(positive_interactions, negative_interactions)))

In [ ]:
def out_shape(shapes):
    return shapes[0]
    

def build_model(num_users: int, num_items: int, latent_dim: int) -> Model:
    """
    Build a model for Bayesian personalized ranking

    :param num_users: a number of the unique users
    :param num_items: a number of the unique movies
    :param latent_dim: vector length for the latent representation
    :return: Model
    """
    user_input = Input((1,), name='user_input')

    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')
    # One embedding layer is shared between positive and negative items
    item_embedding_layer = Embedding(num_items, latent_dim, name='item_embedding', input_length=1)

    positive_item_embedding = Flatten()(item_embedding_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(negative_item_input))

    user_embedding = Embedding(num_users, latent_dim, name='user_embedding', input_length=1)(user_input)
    user_embedding = Flatten()(user_embedding)

    triplet_loss = Lambda(bpr_triplet_loss, output_shape=out_shape)([positive_item_embedding,
                                                             negative_item_embedding,
                                                             user_embedding])

    model = Model(inputs=[positive_item_input, negative_item_input, user_input], outputs=triplet_loss)

    return model

In [ ]:
'''
#1
latent_dim = 32
batch_size = 125 
num_epochs = 1
lr = 0.001
'''
#2
latent_dim = 32
batch_size = 125 
num_epochs = 5 ##
lr = 0.001

'''
#3
latent_dim = 32
batch_size = 125 
num_epochs = 1
lr = 0.01 ##

#4
latent_dim = 32
batch_size = 256 ##
num_epochs = 1
lr = 0.001

#5
latent_dim = 64 ##
batch_size = 256 
num_epochs = 1
lr = 0.001

#6
latent_dim = 32
batch_size = 125 
num_epochs = 10 ##
lr = 0.001

#7
latent_dim = 32
batch_size = 125 
num_epochs = 5
lr = 0.01

#8
latent_dim = 32
batch_size = 125 
num_epochs = 1
lr = 0.1
'''

model = build_model(num_users, num_items, latent_dim)
model.compile(loss=identity_loss, optimizer=Adam(learning_rate=lr))

In [ ]:
trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print('Total number of parameters: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable number of parameters: {:,}'.format(trainable_count))
print('Non-trainable number of parameters: {:,}'.format(non_trainable_count))

print('Training data length: {:,}'.format(df_triplest.shape[0]))

Total number of parameters: 1,888.0
Trainable number of parameters: 1,888
Non-trainable number of parameters: 0.0
Training data length: 3,326


In [ ]:
# fix error Failed to convert a NumPy array to a Tensor (Unsupported object type int)
df_triplest['negative_item_id'] = np.asarray(df_triplest.negative_item_id).astype(np.int64)
df_triplest['positive_item_id'] = np.asarray(df_triplest.positive_item_id).astype(np.int64)

df_triplest.dtypes

positive_item_id    int64
negative_item_id    int64
set_id              int64
dtype: object

In [ ]:
#%%time

X = {
    'user_input': tf.convert_to_tensor(df_triplest.set_id),
    'positive_item_input': tf.convert_to_tensor(df_triplest.positive_item_id),
    'negative_item_input': tf.convert_to_tensor(df_triplest.negative_item_id)
    
}

model.fit(X,tf.ones(df_triplest.shape[0]), batch_size=batch_size,epochs=num_epochs)


Epoch 1/5
27/27 [==============================] - 1s 1ms/step - loss: 0.4984
Epoch 2/5
27/27 [==============================] - 0s 1ms/step - loss: 0.4899
Epoch 3/5
27/27 [==============================] - 0s 2ms/step - loss: 0.4635
Epoch 4/5
27/27 [==============================] - 0s 2ms/step - loss: 0.4072
Epoch 5/5
27/27 [==============================] - 0s 3ms/step - loss: 0.3246


##Evaluation

### Def function

In [ ]:
def full_auc(model: Model, ground_truth: Dict[int, list], items: list) -> float:
    """
    Measure AUC for model and ground truth for all items
    :param model: 
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :return: AUC
    """

    number_of_items = len(items)
    scores = []

    for user_id, true_item_ids in ground_truth:
        predictions = bpr_predict(model, user_id, items)
        grnd = np.zeros(number_of_items, dtype=np.int32)

        for p in true_item_ids:
            index = items.index(p)
            grnd[index] = 1

        if true_item_ids:
          try:
            a=0
            a = roc_auc_score(grnd, predictions)
          except ValueError:
            pass
          scores.append(a)

    return sum(scores) / len(scores)

In [ ]:
def mean_average_precision_k(model: Model, 
                           ground_truth: Dict[int, list], 
                           items: list, 
                           k=100) -> float:
    """
    Calculate mean eavarage precission per user
    
    :param model: 
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :param k: top N recommendations per user
    :return: mean eavarage precission
    """
    scores = []

    for user, actual in ground_truth:
        predictions = bpr_predict(model, user, items)
        predictions = dict(zip(items, predictions))
        predictions = sorted(predictions.items(), key=lambda kv: kv[1], reverse=True)[:k]
        predictions = list(OrderedDict(predictions).keys())

        score = 0.0
        num_hits = 0.0

        for i, p in enumerate(predictions):
            if p in actual:
                num_hits += 1.0
                score += num_hits / (i + 1.0)

        score = score / min(len(actual), k)
        scores.append(score)

    return np.mean(scores)

### prepare ground_truth

In [ ]:
#df_train_split = pd.read_pickle("./Polyvore/cat_v1/train/df_train_split.pkl")
#df_test_split = pd.read_pickle("./Polyvore/cat_v1/test/df_test_split.pkl")

df_train_split = pd.read_pickle("./Polyvore/cat_v2/vgg16/train/df_train_split.pkl")
df_test_split = pd.read_pickle("./Polyvore/cat_v2/vgg16/test/df_test_split.pkl")

In [ ]:
from tqdm import tqdm

In [ ]:
# train
ground_truth_train = pd.DataFrame(columns=['top_id', 'bottom_id'])
data = []
for top_id in tqdm(df_train_split.top_id.unique()):
  b = df_train_split[(df_train_split.top_id == top_id)].bottom_id.unique()

  data.append({'top_id': top_id, 'bottom_id': b})
ground_truth_train = ground_truth_train.append(data, ignore_index=True)



100%|██████████| 29/29 [00:00<00:00, 330.33it/s]


In [ ]:
ground_truth_test = pd.DataFrame(columns=['top_id', 'bottom_id'])
data = []
for top_id in tqdm(df_test_split.top_id.unique()):
  b = df_test_split[(df_test_split.top_id == top_id)].bottom_id.unique()

  data.append({'top_id': top_id, 'bottom_id': b})
ground_truth_test = ground_truth_test.append(data, ignore_index=True)

100%|██████████| 30/30 [00:00<00:00, 737.39it/s]


In [ ]:
ground_truth_train['top_id'] = np.asarray(ground_truth_train.top_id).astype(np.int64)
ground_truth_test['top_id'] = np.asarray(ground_truth_test.top_id).astype(np.int64)

In [ ]:
#Train

##top_id
ground_truth_train['top_id'] = np.asarray(ground_truth_train.top_id).astype(np.int64)
##bottom_id
for i in range(0,len(ground_truth_train.index)):
  a = ground_truth_train['bottom_id'][i]
  ground_truth_train['bottom_id'][i] = a.tolist()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#Test
##top_id
ground_truth_test['top_id'] = np.asarray(ground_truth_test.top_id).astype(np.int64)
##bottom_id
for i in range(0,len(ground_truth_test.index)):
  a = ground_truth_test['bottom_id'][i]
  ground_truth_test['bottom_id'][i] = a.tolist()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
ground_truth_test.dtypes

top_id        int64
bottom_id    object
dtype: object

In [ ]:
import pickle

#ground_truth_train.to_pickle("./Polyvore/cat_v1/train/df_ground_truth_train.pkl")
#ground_truth_test.to_pickle("./Polyvore/cat_v1/test/df_ground_truth_test.pkl")

ground_truth_train.to_pickle("./Polyvore/cat_v2/vgg16/train/df_ground_truth_train.pkl")
ground_truth_test.to_pickle("./Polyvore/cat_v2/vgg16/test/df_ground_truth_test.pkl")

### Test

In [ ]:
#ground_truth_train = pd.read_pickle("./Polyvore/Filtered_top_bottom/train/df_ground_truth_train.pkl")
#ground_truth_test = pd.read_pickle("./Polyvore/Filtered_top_bottom/test/ground_truth_test.pkl")

ground_truth_train = pd.read_pickle("./Polyvore/cat_v2/vgg16/train/df_ground_truth_train.pkl")
ground_truth_test = pd.read_pickle("./Polyvore/cat_v2/vgg16/test/df_ground_truth_test.pkl")

In [ ]:
print(f'AUC train: {full_auc(model, ground_truth_train.values, unique_movie_ids)}')

AUC train: 0.7461570580311906


In [ ]:
print(f'Mean average precision train: {mean_average_precision_k(model, ground_truth_train.values, unique_movie_ids)}')

Mean average precision train: 0.9931820308844622


In [ ]:
print(f'AUC test: {full_auc(model, ground_truth_test.values, unique_movie_ids)}')

AUC test: 0.8405816524339541


In [ ]:
print(f'Mean average precision test: {mean_average_precision_k(model, ground_truth_test.values, unique_movie_ids)}')

Mean average precision test: 0.9094907020313784


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix, mean_squared_error

In [ ]:
def F1_score(model: Model, ground_truth: Dict[int, list], items: list) -> float:
    """
    Measure AUC for model and ground truth for all items
    :param model: 
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :return: AUC
    """

    number_of_items = len(items)
    pred = []
    true = []

    for user_id, true_item_ids in ground_truth:
        predictions = bpr_predict(model, user_id, items)
        grnd = np.zeros(number_of_items, dtype=np.int32)

        for p in true_item_ids:
            index = items.index(p)
            grnd[index] = 1

        pred.append(predictions)
        true.append(grnd)

    preds = pred.nonzero().flatten()

    return f1_score(true, pred, average='weighted')

In [ ]:
print(f'F1 scpre: {F1_score(model, ground_truth_test.values, unique_movie_ids)}')

ValueError: ignored

In [ ]:
if a.any():
  print('Y')
else:
  print('N')

Y


In [ ]:
def sk_evaluate(model, feature, label, label_names):
    pred = model.predict(feature)
    true = np.array(label)

    print('Score on dataset...\n')
    print('Confusion Matrix:\n', confusion_matrix(true, pred))
    print('\nClassification Report:\n', classification_report(true, pred, target_names=label_names))
    print('\naccuracy: {:.3f}'.format(accuracy_score(true, pred)))
    print('f1 score: {:.3f}'.format(f1_score(true, pred, average='weighted')))

    return pred, true

In [ ]:
print('Performance of Mean Word Vector on training dataset...')
_, _ = sk_evaluate(clf, train_X, train_y, label_names=None)

##Make recommendation

In [ ]:
a = [3, 2, 1, 7, 10, 8, 9, 4, 6, 5] #input.top_category

In [ ]:
#full pair_score 
df_pair_score =  pd.DataFrame(columns=['top_id', 'bottom_id','pair_score'])
data = []

for i in range(len(a)) :
  top = a[i]
  pair_score_list =  np.asarray(bpr_predict(model, top, unique_movie_ids)).astype(np.float)
  for j in range(len(pair_score_list)):
    index = j 
    pair_score = pair_score_list[j]
    data.append({'top_id': top,'bottom_id':index, 'pair_score':pair_score})

df_pair_score = df_pair_score.append(data,ignore_index=True)

In [ ]:
df_pair_score.head(5)

In [ ]:
# top 5:
top_5_pairs =  pd.DataFrame(columns=['pair_score', 'top_id','bottom_id'])
df_top5 =  pd.DataFrame(columns=['pair_score', 'top_id','bottom_id'])

for top_id in df_pair_score.top_id.unique():
  tmp = df_pair_score.loc[(df_pair_score['top_id'] == top_id)]
  data = sorted(zip(tmp.pair_score,tmp.top_id,tmp.bottom_id), reverse=True)[:5]
  del df_top5
  df_top5 = pd.DataFrame(data,columns=['pair_score', 'top_id','bottom_id'])
  top_5_pairs = top_5_pairs.append(df_top5,ignore_index= True)

In [ ]:
#test
#df_pair_score.loc[(df_pair_score.top_id==2)].sort_values(by=['pair_score'],ascending = False)
top_5_pairs.head(5)

In [ ]:
best_match_bottom = top_5_pairs.bottom_id.unique()

In [ ]:
best_match_bottom

In [ ]:
# img list with its attributes: img_id, bottom_category
cat = bottom['new']
for every img_id:
  for every bottom_id in bottom_category:
    score = 0
    for i in len(cat):
    if bottom_id == cat[i]:
      
      score = score +

In [ ]:
cat = bottom['new']

## Make RCM 2:

In [ ]:
#top= pd.read_pickle("./Polyvore/cat_v1/top_categories.pkl")

top = pd.read_pickle(("./Polyvore/cat_v2/vgg16/top_categories.pkl"))
bottom = pd.read_pickle(("./Polyvore/cat_v2/vgg16/bottom_categories.pkl"))

### Table with full list of pair score:

In [ ]:
#table with full list of pair score:

df_pair_score =  pd.DataFrame(columns=['top_id', 'bottom_id','pair_score'])
data = []
a = top['new']

for i in range(len(a)) :
  top = a[i]
  pair_score_list =  np.asarray(bpr_predict(model, top, unique_movie_ids)).astype(np.float)
  for j in range(len(pair_score_list)):
    index = j 
    pair_score = pair_score_list[j]
    data.append({'top_id': top,'bottom_id':index, 'pair_score':pair_score})

df_pair_score = df_pair_score.append(data,ignore_index=True)

In [ ]:
df_pair_score

,top_id,bottom_id,pair_score
0,0,0,0.022888
1,0,1,0.000591
2,0,2,0.025932
3,0,3,0.017773
4,0,4,0.016789
...,...,...,...
865,29,24,0.044797
866,29,25,0.053655
867,29,26,0.041138
868,29,27,0.059279


###Table with full bottom_list + its categories

In [ ]:
y = df_reg.loc[df_reg['cate1_name'] != 'upper-body', ('id','cate2_name','texture_name','sleeve_part_name','shape_name','neckline_part_name','fabric_name','style_name')]
#dont use attribute columns that only have 1 value
y['atts'] = y[['cate2_name','texture_name','sleeve_part_name','shape_name','neckline_part_name','fabric_name','style_name']].values.tolist()

In [ ]:
# map raw category to number category
bottom_tmp = pd.DataFrame(columns =['img_id','bottom_category'])
data = []

for id in tqdm(y.id.unique()):
  a = y[(y.id == id)].atts.tolist()[0]
  for i in range(0,len(a)):
    for j in range(0,len(bottom.index)):
      if a[i] == bottom['old'][j]:
        b = bottom['new'][j]
        data.append({'img_id':id,'bottom_category':b})
bottom_tmp = bottom_tmp.append(data,ignore_index = True)


100%|██████████| 11333/11333 [00:37<00:00, 302.54it/s]


In [ ]:
#group by img_id
bottom_img = pd.DataFrame(columns =['img_id','bottom_category'])
data = []

for img_id in tqdm(bottom_tmp.img_id.unique()):
  bt = bottom_tmp[(bottom_tmp.img_id == img_id)].bottom_category.unique()
  data.append({'img_id':img_id, 'bottom_category':bt})

bottom_img = bottom_img.append(data,ignore_index = True)

In [ ]:
bottom_img

In [ ]:
df_pair_score.to_pickle("./Polyvore/cat_v2/vgg16/df_pair_score.pkl")
bottom_img.to_pickle("./Polyvore/cat_v2/vgg16/bottom_img.pkl")

### Table with input top image's information

In [ ]:
top = pd.read_pickle("./Polyvore/cat_v2/vgg16/top_categories.pkl")

In [ ]:
x = df_reg.loc[df_reg['cate1_name'] == 'upper-body', ('id','cate2_name','texture_name','sleeve_part_name','shape_name','neckline_part_name','fabric_name','style_name')].head(30)
#dont use attribute columns that only have 1 value
# CHANGE TO ACTUAL INPUT TOP, 
x['atts'] = x[['cate2_name','texture_name','sleeve_part_name','shape_name','neckline_part_name','fabric_name','style_name']].values.tolist()

In [ ]:
# map raw category to number category
top_tmp = pd.DataFrame(columns =['img_id','top_category'])
data = []

for id in tqdm(x.id.unique()):
  a = x[(x.id == id)].atts.tolist()[0]
  for i in range(0,len(a)):
    for j in range(0,len(top.index)):
      if a[i] == top['old'][j]:
        b = top['new'][j]
        data.append({'img_id':id,'top_category':b})
top_tmp = top_tmp.append(data,ignore_index = True)

100%|██████████| 30/30 [00:00<00:00, 462.50it/s]


In [ ]:
#group by img_id
input_top_full = pd.DataFrame(columns =['img_id','top_category'])
data = []

for img_id in tqdm(top_tmp.img_id.unique()):
  t = top_tmp[(top_tmp.img_id == img_id)].top_category.unique()
  data.append({'img_id':img_id, 'top_category':t})

input_top_full = input_top_full.append(data,ignore_index = True)

100%|██████████| 30/30 [00:00<00:00, 1392.78it/s]


In [ ]:
input_top_full.to_pickle("./Polyvore/cat_v2/vgg16/top_img_30.pkl")

In [ ]:
input_top_full

,img_id,top_category
0,214181831/1.jpg,"[0, 1, 2, 3, 4, 5, 6]"
1,214181831/2.jpg,"[0, 7, 2, 3, 4, 8, 6]"
2,214181831/7.jpg,"[0, 9, 2, 3, 4, 5, 6]"
3,120161271/1.jpg,"[0, 7, 10, 3, 4, 5, 6]"
4,143656996/1.jpg,"[0, 1, 11, 3, 12, 5, 6]"
5,143656996/2.jpg,"[0, 1, 2, 3, 4, 5, 6]"
6,186627934/1.jpg,"[0, 1, 2, 3, 4, 5, 6]"
7,186627934/3.jpg,"[0, 7, 11, 3, 4, 5, 6]"
8,206969379/1.jpg,"[0, 1, 11, 3, 12, 5, 6]"
9,206969379/2.jpg,"[0, 1, 11, 3, 12, 5, 6]"


In [ ]:
#get 1 image to run the recommendation
input_top = input_top_full#.iloc[[1,2,3,4,5,6,7,8,9,10]]
input_top

,img_id,top_category
0,214181831/1.jpg,"[0, 1, 2, 3, 4, 5, 6]"
1,214181831/2.jpg,"[0, 7, 2, 3, 4, 8, 6]"
2,214181831/7.jpg,"[0, 9, 2, 3, 4, 5, 6]"
3,120161271/1.jpg,"[0, 7, 10, 3, 4, 5, 6]"
4,143656996/1.jpg,"[0, 1, 11, 3, 12, 5, 6]"
5,143656996/2.jpg,"[0, 1, 2, 3, 4, 5, 6]"
6,186627934/1.jpg,"[0, 1, 2, 3, 4, 5, 6]"
7,186627934/3.jpg,"[0, 7, 11, 3, 4, 5, 6]"
8,206969379/1.jpg,"[0, 1, 11, 3, 12, 5, 6]"
9,206969379/2.jpg,"[0, 1, 11, 3, 12, 5, 6]"


In [ ]:
top_category = input_top.top_category[0]
top_category

array([0, 1, 2, 3, 4, 5, 6], dtype=object)

### Calculate bottom_img_score

In [ ]:
for a in range(0,len(input_top)):
  top_category= input_top.top_category[a]
  data =[]
  output_bottom = pd.DataFrame(columns = ['img_id','img_score','input_top'])

  for img_id in tqdm(bottom_img.img_id.unique()):
    bottom_category = bottom_img[(bottom_img.img_id == img_id)].bottom_category.values[0]
    score = 0
    img_score = 0
    #top_category = input_top.top_category[0]
    for i in range(0,len(bottom_category)):
      for j in range(0,len(top_category)):
        x = df_pair_score[(df_pair_score.top_id == top_category[j])&(df_pair_score.bottom_id == bottom_category[i])].pair_score.values
        score = score + x
    data.append ({'img_id':img_id,'img_score':score,'input_top':top_category})

  output_bottom = output_bottom.append(data,ignore_index=True )


In [ ]:
a = {'a':[1,2,3,4,5,6],'b':['a','b','c','a','a','a'],'c':['aa','bb','cc','aa','aa','aa']}

In [ ]:
df = pd.DataFrame(data=a)
df

,a,b,c
0,1,a,aa
1,2,b,bb
2,3,c,cc
3,4,a,aa
4,5,a,aa
5,6,a,aa


In [ ]:
output_bottom.sort_values(['img_id','img_score','input_top'],ascending=False).groupby('img_score').head(1)

,a,b,c
5,6,a,aa
2,3,c,cc
1,2,b,bb


In [ ]:
output_bottom.groupby("input_top").sort_values(by='img_score')

In [ ]:
top_category[0]

array([0, 1, 2, 3, 4, 5, 6], dtype=object)

In [ ]:
#top_category = np.asarray(input_top.top_category.tolist(),dtype=object)
top_category = input_top.top_category[0]
data =[]
output_bottom = pd.DataFrame(columns = ['img_id','img_score'])

for img_id in tqdm(bottom_img.img_id.unique()):
  bottom_category = bottom_img[(bottom_img.img_id == img_id)].bottom_category.values[0]
  score = 0
  img_score = 0
  #top_category = input_top.top_category[0]
  for i in range(0,len(bottom_category)):
    for j in range(0,len(top_category)):
      x = df_pair_score[(df_pair_score.top_id == top_category[j])&(df_pair_score.bottom_id == bottom_category[i])].pair_score.values
      score = score + x
  data.append ({'img_id':img_id,'img_score':score})

output_bottom = output_bottom.append(data,ignore_index=True )




In [ ]:
output_bottom

In [ ]:
suggest = output_bottom.sort_values(by=['img_score'],ascending = False)
suggest.head(10)

,img_id,img_score
9640,151337160/1.jpg,[276.41446144506335]
10280,120736583/3.jpg,[276.41446144506335]
6114,150060917/1.jpg,[276.41446144506335]
10276,173808693/3.jpg,[276.41446144506335]
6109,210706200/1.jpg,[276.41446144506335]
6108,195070979/2.jpg,[276.41446144506335]
2458,213029175/3.jpg,[276.41446144506335]
6096,211911509/2.jpg,[276.41446144506335]
2460,187552785/3.jpg,[276.41446144506335]
6090,215833068/1.jpg,[276.41446144506335]


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
import cv2
img_path ='./Polyvore/images/images/143656996/1.jpg'
image = cv2.imread(img_path)
#crop_img = img[y:y+h, x:x+w]
cv2_imshow(image)
cv2.waitKey(0)

#143656996/1

AttributeError: ignored

In [ ]:
list = pd.read_csv('./output_bottom_sorted.csv')

In [ ]:
a =pd.read_pickle('/content/drive/MyDrive/Practicum- ChauDao/Polyvore/cat_v2/vgg16/output_bottom_sorted.pkl')

In [ ]:
a = a.set_index('img_id')

In [ ]:
a.to_csv('/content/drive/MyDrive/Practicum- ChauDao/Polyvore/cat_v2/vgg16/output_bottom_sorted.csv')

In [ ]:
output_bottom_sorted = pd.read_pickle("./Polyvore/cat_v2/vgg16/output_bottom_sorted.pkl")

In [ ]:
output_bottom_sorted

,img_id,img_score,input_top,img_score_num
30987,216900310/1.jpg,[3.4903855475131422],214181831/7.jpg,3.490386
18529,217030410/2.jpg,[7.304534606169909],214181831/2.jpg,7.304535
9561,212540254/1.jpg,[0.1731702722609043],214181831/1.jpg,0.173170
104491,217013995/8.jpg,[6.307556576561183],206969379/2.jpg,6.307557
93158,217013995/8.jpg,[6.307556576561183],206969379/1.jpg,6.307557
88371,217026804/3.jpg,[6.282377630006522],186627934/3.jpg,6.282378
77559,212540254/1.jpg,[0.1731702722609043],186627934/1.jpg,0.173170
66226,212540254/1.jpg,[0.1731702722609043],143656996/2.jpg,0.173170
47826,217013995/8.jpg,[6.307556576561183],143656996/1.jpg,6.307557
36493,217013995/8.jpg,[7.5537522728554904],120161271/1.jpg,7.553752


In [ ]:
df_pair_score=pd.read_pickle("./Polyvore/cat_v2/vgg16/df_pair_score.pkl")
bottom_img= pd.read_pickle("./Polyvore/cat_v2/vgg16/bottom_img.pkl")
top_img= pd.read_pickle("./Polyvore/cat_v2/vgg16/top_img_30.pkl")

In [ ]:
df_pair_score.to_csv("./Polyvore/cat_v2/vgg16/df_pair_score.csv")
bottom_img.to_csv("./Polyvore/cat_v2/vgg16/bottom_img.csv")
top_img.to_csv("./Polyvore/cat_v2/vgg16/top_img_30.csv")

In [ ]:
top_img

,img_id,top_category
0,214181831/1.jpg,"[0, 1, 2, 3, 4, 5, 6]"
1,214181831/2.jpg,"[0, 7, 2, 3, 4, 8, 6]"
2,214181831/7.jpg,"[0, 9, 2, 3, 4, 5, 6]"
3,120161271/1.jpg,"[0, 7, 10, 3, 4, 5, 6]"
4,143656996/1.jpg,"[0, 1, 11, 3, 12, 5, 6]"
5,143656996/2.jpg,"[0, 1, 2, 3, 4, 5, 6]"
6,186627934/1.jpg,"[0, 1, 2, 3, 4, 5, 6]"
7,186627934/3.jpg,"[0, 7, 11, 3, 4, 5, 6]"
8,206969379/1.jpg,"[0, 1, 11, 3, 12, 5, 6]"
9,206969379/2.jpg,"[0, 1, 11, 3, 12, 5, 6]"
